<a href="https://colab.research.google.com/github/slng09/advance-task22/blob/main/schoolcleanfile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

#--- suppress warnings ---
import warnings
warnings.filterwarnings("ignore")

In [71]:
# Read CSV
from google.colab import files
uploaded = files.upload()
import io
df = pd.read_csv(io.BytesIO(uploaded["Victoria.csv"]),sep=',')
df.head()

Saving Victoria.csv to Victoria (6).csv


,School_No,School_Name,School_Type,Address_Line_1,Address_Line_2,Address_Town,Address_State,Address_Postcode,Postal_Address_Line_1,Postal_Address_Line_2,Postal_Postcode,Full_Phone_No,LGA_ID,LGA_Name,X,Y,Education_Sector,Entity_Type
0,1,Alberton Primary School,Primary,21 Thomson Street,NaN,Alberton,VIC,3971,21 Thomson Street,NaN,3971,03 5183 2412,681,Wellington (S),146.666601,-38.617713,Government,1
1,3,Allansford and District Primary School,Primary,Frank Street,NaN,Allansford,VIC,3277,Frank Street,NaN,3277,03 5565 1382,673,Warrnambool (C),142.590393,-38.386281,Government,1
2,4,Avoca Primary School,Primary,118 Barnett Street,NaN,Avoca,VIC,3467,P O Box 12,NaN,3467,03 5465 3176,599,Pyrenees (S),143.475649,-37.084502,Government,1
3,8,Avenel Primary School,Primary,40 Anderson Street,NaN,Avenel,VIC,3664,40 Anderson Street,NaN,3664,03 5796 2264,643,Strathbogie (S),145.234722,-36.901368,Government,1
4,12,Warrandyte Primary School,Primary,5-11 Forbes Street,NaN,Warrandyte,VIC,3113,5-11 Forbes Street,NaN,3113,03 9844 3537,421,Manningham (C),145.213980,-37.742675,Government,1


In [54]:
df.isnull().sum()

School_No                   0
School_Name                 0
School_Type                 0
Address_Line_1              0
Address_Line_2           2235
Address_Town                0
Address_State               0
Address_Postcode            0
Postal_Address_Line_1       0
Postal_Address_Line_2    2235
Postal_Postcode             0
Full_Phone_No               2
LGA_ID                      0
LGA_Name                    0
X                           0
Y                           0
Education_Sector            0
Entity_Type                 0
dtype: int64

In [ ]:
def clean_column_name(column_names):
    clean_column_names = []
    for c in column_names:
        c = c.lower().replace(' ','_')
        clean_column_names.append(c)
    return clean_column_names    

df.columns = clean_column_name(df.columns)

In [73]:
# Merge the null column and the text column into a single column
df['Address'] = df['Address_Line_1'].fillna('') + df['Address_Line_2'].fillna('')
df['Postal_Address'] = df['Postal_Address_Line_1'].fillna('') + df['Postal_Address_Line_2'].fillna('')
df.Address

0             21 Thomson Street
1                  Frank Street
2            118 Barnett Street
3            40 Anderson Street
4            5-11 Forbes Street
                 ...           
2243          35 Sturdee Street
2244    Level 1117 Capel Street
2245     95-97 Brunswick Street
2246        164 - 180 Kings Way
2247          425 Riversdale Rd
Name: Address, Length: 2248, dtype: object

In [78]:
#change column names
df = df.drop(['Address_Line_1','Address_Line_2','Postal_Address_Line_1', 'Postal_Address_Line_2', 'Postal_Postcode', 'Address', 'Postal_Address'],axis = 1)
df.columns

Index(['School_No', 'School_Name', 'School_Type', 'Address_Town',
       'Address_State', 'Address_Postcode', 'Full_Phone_No', 'LGA_ID',
       'LGA_Name', 'X', 'Y', 'Education_Sector', 'Entity_Type', 'sum_column'],
      dtype='object')

In [79]:
df.columns= ['School_No', 'School_Name', 'School_Type', 'Address_Town',
       'Address_State', 'Address_Postcode', 'Full_Phone_No', 'LGA_ID',
       'LGA_Name', 'Latitude', 'Longtitude', 'Education_Sector', 'Entity_Type', 'Address']
df.columns

Index(['School_No', 'School_Name', 'School_Type', 'Address_Town',
       'Address_State', 'Address_Postcode', 'Full_Phone_No', 'LGA_ID',
       'LGA_Name', 'Latitude', 'Longtitude', 'Education_Sector', 'Entity_Type',
       'Address'],
      dtype='object')

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2248 entries, 0 to 2247
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   School_No         2248 non-null   int64  
 1   School_Name       2248 non-null   object 
 2   School_Type       2248 non-null   object 
 3   Address_Town      2248 non-null   object 
 4   State             2248 non-null   object 
 5   Postcode          2248 non-null   int64  
 6   Full_Phone_No     2246 non-null   object 
 7   LGA_ID            2248 non-null   int64  
 8   LGA_Name          2248 non-null   object 
 9   Latitude          2248 non-null   float64
 10  Longtitude        2248 non-null   float64
 11  Education_Sector  2248 non-null   object 
 12  Entity_Type       2248 non-null   int64  
 13  Address           2248 non-null   object 
 14  Postal_Address    2248 non-null   object 
dtypes: float64(2), int64(4), object(9)
memory usage: 263.6+ KB


In [59]:
duplicate = df[df.duplicated()]
print("Duplicate Rows :")
duplicate

Duplicate Rows :


,School_No,School_Name,School_Type,Suburb,State,Postcode,Full_Phone_No,LGA_ID,LGA_Name,Latitude,Longtitude,Education_Sector,Entity_Type,Address,Postal_Address


In [76]:
# Compare the two columns and drop duplicates
#df['sum_column'] = df['Address'].map(str) +' '+ df['Postal_Address']

# Remove duplicates from the sum column
df['sum_column'] = df.apply(lambda x: x['Address'] +' '+ x['Postal_Address'] if x['Address'] != x['Postal_Address'] else x['Address'], axis=1)

# Print the resulting dataframe
df.sum_column

0                   21 Thomson Street
1                        Frank Street
2       118 Barnett Street P O Box 12
3                  40 Anderson Street
4                  5-11 Forbes Street
                    ...              
2243     35 Sturdee Street PO Box 510
2244          Level 1117 Capel Street
2245           95-97 Brunswick Street
2246              164 - 180 Kings Way
2247                425 Riversdale Rd
Name: sum_column, Length: 2248, dtype: object

In [68]:
df = df.drop(['Address','Postal_Address'],axis = 1)
df.columns

Index(['School_No', 'School_Name', 'School_Type', 'Suburb', 'State',
       'Postcode', 'Full_Phone_No', 'LGA_ID', 'LGA_Name', 'Latitude',
       'Longtitude', 'Education_Sector', 'Entity_Type', 'sum_column'],
      dtype='object')

In [69]:
df.columns=['School_No', 'School_Name', 'School_Type', 'Suburb', 'State',
       'Postcode', 'Full_Phone_No', 'LGA_ID', 'LGA_Name', 'Latitude',
       'Longtitude', 'Education_Sector', 'Entity_Type', 'Address']
df.columns

Index(['School_No', 'School_Name', 'School_Type', 'Suburb', 'State',
       'Postcode', 'Full_Phone_No', 'LGA_ID', 'LGA_Name', 'Latitude',
       'Longtitude', 'Education_Sector', 'Entity_Type', 'Address'],
      dtype='object')

In [21]:
#--- take the means to replace the zero 

df['Postal_Address_Line_2'] = df['Postal_Address_Line_2'].replace([np.nan], ['-'])

In [ ]:
address_list = df['Address'].to_list() 
latitude = df['Latitude'].to_list()
longitude = df['Longitude'].to_list()

geolocator = Nominatim(user_agent="test")

def reverse_geocoding(lat, lon):
    try:
        location = geolocator.reverse(f"{lat}, {lon}")
        return location.raw['display_name']
    except:
        return None
    
# Find rows in df with a missing address
df_nan = df[df.Address.isnull()]

# Reverse geocode the latitude and longitude to get the missing addresses
addresses = np.vectorize(reverse_geocoding)(df_nan['Latitude'], df_nan['Longitude'])

# Update the Address column in df_nan with the reverse geocoded addresses
df_nan['Address'] = addresses

# Update df with the modified df_nan
df.update(df_nan)

df_nan

In [ ]:
name_list = df['Name'].to_list() 
locator = Nominatim(user_agent="my-application", timeout=20)

def get_address(name):
    """Get the address of a location using its name"""
    try:
        location = locator.geocode(f"{name}")
        return location.raw['display_name']
    except:
        return None
        
# Find rows in df with a missing address
df_nan = df[df.Address.isnull()]

# Get the missing addresses
addresses = df_nan['Name'].apply(get_address)

# Update the Address column in df_nan with the missing addresses
df_nan['Address'] = addresses

# Update df with the modified df_nan
df.update(df_nan)

df_nan


In [80]:
df.isnull().sum()

School_No           0
School_Name         0
School_Type         0
Address_Town        0
Address_State       0
Address_Postcode    0
Full_Phone_No       2
LGA_ID              0
LGA_Name            0
Latitude            0
Longtitude          0
Education_Sector    0
Entity_Type         0
Address             0
dtype: int64

In [ ]:
# Replace NaN with zero on all columns 
df = df.fillna(0)

In [81]:
from google.colab import files
df.to_csv('Victoria.csv') 
files.download('Victoria.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>